# Measuring Land Productivity with Harmonized Landsat and Sentinel-2: A Cloud Native Approach

The cloud native geospatial paradigm has the potential to make Earth observation analysis accessible to more people, more easily.
In the simplest terms, instead of downloading data before performing an analysis, it’s now possible to stream data directly from the cloud.

This notebook walks through the steps required to query and load NASA's Harmonized Landsat and Sentinel-2 (HLS) data using a Spatio-Temportal Asset Catalog (STAC), and how to use that data to investigate land productivity.
Importantly, the benefit of using a cloud native approach is that there is no requirement to download and process individual scenes. 
Instead, a query is used to identify the relevant scenes, and then software is used to only load a relevant portion of those scenes into the computer's memory. 

## Use case: measuring land productivity for SDG Indicator 15.3.1

Sustainable Development Goal 15 is to "Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, and halt and reverse land degradation and halt biodiversity loss".
Under this goal is Target 15.3, which stipulates "By 2030, combat desertification, restore degraded land and soil, including land affected by desertification, drought and floods, and strive to achieve a land degradation-neutral world".
The relevant indicator is 15.3.1: "Proportion of land that is degraded over total land area", which has three sub-indicators: trends in land cover, trends in land productivity or functioning, and trends in carbon stocks above and below ground. 

![Flowchart demonstrating that three indicators make up SDG indicator 15.3.1](images/steps_for_indicator_1531.png "SDG Indicator 15.3.1 Methodology")

*Figure 1. Steps to derive the indicator from the sub-indicators where ND is not degraded and D is degraded. Source: [Good practice guidence. SDG Indicator 15.3.1](https://www.unccd.int/resources/manuals-and-guides/good-practice-guidance-sdg-indicator-1531-proportion-land-degraded).*

Earth observation data has a strong role to play in all three sub-indicators, and regular remote sensing of the environment from satellites can assist with looking at trends in both land cover and land productivity. 
As a demonstration, this notebook focusses on the land productivity subindicator, rather than the full 15.3.1 indicator.
For more information, we recommend the following resources:

* [Good practice guidance. SDG indicator 15.3.1, Proportion of land that is degraded over total land area. Version 2.0.](https://www.unccd.int/resources/manuals-and-guides/good-practice-guidance-sdg-indicator-1531-proportion-land-degraded)
* [Satellite Data Requirements for SDGIndicator 15.3.1](https://ceos.org/sdg/files/supportsheets/SDG_15.3.1_EO_Satellite_Data_Requirements_31Aug2022.pdf)
* [TRENDS.EARTH: tracking land change](https://maps.trends.earth/map?tab=layers&zoom=7&center=lat%3D-8.477805461808186%26lng%3D-67.87353515625001&layers=%5B%5D&basemap=satellite)


## Application area

This notebook will calculate the trend in land productivity for a forested region in Capão Bonito, a municipality in the state of São Paulo in Brazil. 
Today, there are large areas of forest in the sourthern region, but this wasn't always the case. 
The video below shows the region from 1984 through to today, with large areas of grasslands converting to forests around 2008-2012.

<iframe width="1080" height="600" src="https://earthengine.google.com/iframes/timelapse_player_embed.html#v=-24.11033,-48.2946,11.685,latLng&t=3.45&ps=50&bt=19840101&et=20221231" frameborder="0" allowfullscreen></iframe>

This notebook uses NASA's Harmonized Landsat Sentinel-2 product, which provides imagery from 2013 onwards. 
As such, the purpose of this notebook is to examine whether the trend in land productivity has been stable since the reforesting.

## Notebook overview

This notebook demonstrates how to use Earth observation data from satellites to measure the trend in land productivity for a region. The notebook is broken into three parts, as follows.

### Part 1: Querying and loading

1. Setting up the notebook
2. Querying NASA's Earthdata STAC catalog to identify relevent scenes from the last year
3. Loading data from the identified scenes

### Part 2: Preparing data and identifying the peak productivity period

1. Removing clouds from the loaded data
2. Measuring land productivity per pixel using a vegetation index as a proxy
3. Calculating the maximum vegetation index value for each month
4. Identifying the period of peak productivity by looking at the average maximum value for the area in each month

### Part 3: Land productivity trend from 2013 to 2024

1. Querying NASA's Earthdata STAC to identify relevent scenes from the peak productivity period of each year since 2013
2. Loading the data from the identified scenes
3. Applying cloud masking and calculating the vegetation index
4. Measuring the maximum vegetation index value for the peak period of each year
5. Observing the change in maximum vegetation index value over the last ten years

## Part 1: Querying and loading

### 1.1 Setting up the notebook

The first step is to set up the requried Python libraries and local imports.

* `json` is used to load your Earthdata token
* `odc.stac` and `pystac_client` are used to access the NASA Earthdata STAC
* `geopandas` and `numpy` are used to manipulate data
* `utils` is a local module that contains a configuration dictionary for Harmonized Landsat and Sentinel-2 (HLS)

In [ ]:
import json

from odc.stac import configure_rio, load
from pystac_client import Client as PystacClient
import geopandas as gpd
import numpy as np

from utils import hls_config

The second step is to start a Dask client.

Dask supports local parallel processing and can help speed up computation times.

In [ ]:
from dask.distributed import Client as DaskClient

dask_client = DaskClient()
dask_client

### 1.2 Querying NASA's CMR STAC catalog to identify relevent scenes from the last year

NASA's Earthdata STAC catalog contains structured metadata that can be used by software to find relevant Landsat and Sentinel-2 scenes that match a user's requested area of interest and time frame.

The first step is to specify the catalog and then connect to it.

In [ ]:
# The catalog URL for the Earthdata STAC containing HLS
catalog = "https://cmr.earthdata.nasa.gov/cloudstac/LPCLOUD/"

# pystac_client is used to connect to the catalog
stac_client = PystacClient.open(catalog)

The next step is to specify which collections to connect to.
In this case, HLS is made up of two collections:

* Harmonized Landsat at 30m resolution: `HLSL30.v2.0`
* Harmonized Sentinel-2 at 30m resolution: `HLSS30.v2.0`

In [ ]:
# Specify both Landsat and Sentinel-2 HLS collections for the query
collections = ["HLSL30.v2.0", "HLSS30.v2.0"]

The next step is to specify an area of interest, a forested region in Capão Bonito, a municipality in the state of São Paulo in Brazil. 

This notebook uses geopandas to read the area of interest as a polygon from a GeoJSON file. 

* `aoi` is the full GeoDataFrame
* `aoi_geom` is the polygon of the first item in the GeoDataFrame.

In [ ]:
# Get the area of interest as a GeoDataFrame using geopandas
aoi = gpd.read_file("aoi.geojson")

# Separate out the polygon for the first item in the GeoDataFrame
aoi_geom = aoi.iloc[0].geometry

# View the area of interest on an interactive map
aoi.explore()

The next step is to specify a date range. 

For part 1 and part 2, the notebook will use one year worth of data.

* `start_date` and `end_date` are provided as YYYY-MM-DD strings
* `date_range` is a string used in the query, which has the start and end dates separated by `/`

In [ ]:
# Set the start date and end date
start_date = "2022-06-01"
end_date = "2023-06-01"

# Format the date range for the query
date_range = f"{start_date}/{end_date}"

The final step is to use the `client` connection to connect to the Earthdata STAC and search for all Landsat and Sentinel-2 scenes that intersect with the area of interest polygon within the specified date range.
In the terminology of STAC, each scene is an `item`.

Once complete, the code will print out the number of items that matched the query parameters.

In [ ]:
# Search for items in the collection
items = list(
    stac_client.search(
        collections=collections, intersects=aoi_geom, datetime=date_range
    ).items()
)

print(f"Found {len(items)} items")

### 1.3 Loading data from the identified scenes

After finding the relevant STAC items, it is now possible to load the relevant data directly from those items.

The first step is to set up authentication to allow access to the data contained within NASA's Earthdata STAC. 
Until this point, the notebook has only queried the metadata stored in the STAC.

The [earthaccess](https://github.com/nsidc/earthaccess) library wraps common methods for authorizing, searching and accessing data on NASA's Earthdata cloud. Before using earthaccess, you need to register for an account with NASA's Earthdata system. The next line will prompt you for your username and password (at the top of the browser if using Github Codespaces).

In [ ]:
import earthaccess

earthaccess.login()

We extract just the token from our logged in `earthaccess` instance so we can use it with rasterio via odc.stac.

In [ ]:
# This code will be wrapped by `get_edl_token` in the next earthaccess release
token = earthaccess.__auth__.token["access_token"]

# Configure rasterio to use cloud defaults, and GDAL to use the authorization token
header_string = f"Authorization: Bearer {token}"
configure_rio(cloud_defaults=True, GDAL_HTTP_HEADERS=header_string)

The next step is to use `odc.stac` to lazily load the queried data. 
Here, lazy loading is provided by Dask and means that it is possible to see information about the dimensionality and type of data, without pulling all the numerical satellite data. 

The `load` command from `odc.stac` has a number of arguments:
* `items` are the scenes identified by querying NASA's Earthdata STAC
* `resolution` specifies the resolution (in metres) for the loaded data
* `crs` specifies the coordinate reference system for the loaded data
* `chunks={}` specifies that lazy loading should be performed
* `groupby="solarday"` specifies that scenes that occur on the same day should be grouped
* `stac_cfg` specifies any additional configuration requried to load the data. See the `utils.py` file for details
* `bands` specifies the spectral and quality bands to load
* `geopolygon` specifies the area of interest to load

When the lazy load is complete, a nicely formatted summary of the lazy-loaded data will appear

In [ ]:
data = load(
    items,
    resolution=30,
    crs="EPSG:5530",
    chunks={},
    groupby="solar_day",
    stac_cfg=hls_config,
    bands=["red", "green", "blue", "nir", "fmask"],
    geopolygon=aoi_geom,
)

data

The final step is to have a quick look at the data using a custom function for making a true color image. 
Any visualisation will require data to be loaded into memory, so the following step will take a moment to read the necessary data.

In [ ]:
# Define a simple plotting function to reuse
def plot_rgb(data):
    # Select the red green and blue bands
    rgb = data[["red", "green", "blue"]]

    # Select a subset of images that show clear and cloudy images
    rgb_subset = rgb.isel(time=slice(4, 7)).to_array()

    # Display the image
    rgb_subset.plot.imshow(col="time", col_wrap=3, vmin=0, vmax=2000)


plot_rgb(data)

## Part 2: Preparing data and identifying the peak productivity period

### 2.1 Removing clouds from the loaded data

When displaying three timesteps of the loaded data in the previous section, it was evident that some images contain clouds.
Rather than throw cloud-affected images away, it is possible to use quality information (provided in the `fmask` band) to remove the offending pixels. 

For this, a pixel quality mask must be constructed that identifies the locations of cloud and cloud shadow. The `fmask` band contains integer values that encode the quality of the pixel using bit-flags, as shown in Figure 2.

![Diagram showing which bits map to each pixel quality flag](images/cloudmask.jpg "Bit-masking using fmask")

*Figure 2: Bits are read from right to left. A value of 1 indicates True, a value of 0 indicates False. In this case, the binary digit will match against any pixels that are classified as cloud, adjacent cloud, or cloud shadow. For more information see the [HLS User Guide](https://lpdaac.usgs.gov/documents/1698/HLS_User_Guide_V2.pdf)*

The following cell uses a number of variables:

* `pq_bin` is the binary digit with values of `1` for cloud, adjacent cloud, and cloud shadow
* `pq_mask` is a True/False array with a value of True if the `fmask` value has flags that match any of the `1` flags in `pq_bin`
* `nodata_mask` is a True/False array with a value of True if the `fmask` value matches the no data value
* `mask` is a True/False array with a value of True if the pixel belongs to either the `pq_mask` or the `nodata_mask`

In [ ]:
# Write a function to reuse
def apply_quality_mask(data):
    # Specify pixel quality mask by setting the binary flags
    # bit 1 (cloud), 2 (adjacent cloud), and 3 (cloud shadow) to True
    pq_bin = 0b00001110
    pq_mask = (data.fmask & pq_bin) != 0

    # Specify the no data mask as any pixel that is equal to the nodata value in the metadata
    nodata_mask = data.fmask == data.fmask.odc.nodata

    # Combine and apply both masks
    # If the mask is True, replace the pixel value with nan
    mask = pq_mask | nodata_mask
    masked = data.where(~mask, other=np.nan)
    masked.drop_vars(["fmask"])

    return masked


masked = apply_quality_mask(data)

The final step is to plot the masked data to see the effect of the masking.

In [ ]:
# Show the masked data
plot_rgb(masked)

### 2.2 Measuring land productivity per pixel using a vegetation index as a proxy

For SDG Indicator 15.3.1, a vegetation index is used as a proxy for land productivity. 

The first step is to apply scaling to the relevant bands -- HLS values are supplied as integers, but it is best practice to scale them to have values between 0 and 1 when doing calculations. HLS has a scale factor of `0.0001`. The cell below defines a function for this, and also sets any negative values to `nan`.

After that, a vegetation index can be calculated. 
There are multiple vegetation indices that can be used for land productivity. 
This example uses the two-band enhanced vegetation index (EVI2), which is preferred for regions of high biomass.

$$\text{EVI2} = 2.4 \times \frac{\text{NIR} - \text{Red}}{\text{NIR} + \text{Red} + 1}$$

The index has values between -1 and 1, with values closer to 1 indicating dense, productive vegetation.

In [ ]:
def scale_offset(band):
    # Apply a scaling factor of 0.0001 and only keep values greater than 0
    band = band * 0.0001
    band = band.where(band > 0, other=np.nan)

    return band


def calculate_evi2(data):
    nir = scale_offset(data.nir)
    red = scale_offset(data.red)

    evi2 = 2.4 * ((nir - red) / (nir + red + 1))

    return evi2


masked["evi2"] = calculate_evi2(masked)

The next step is to plot the calculated index.
The next cell defines a function that displays the same time steps used for the true color image.

In [ ]:
def plot_evi2(data):
    data.evi2.isel(time=slice(4, 7)).plot.imshow(
        col="time", col_wrap=3, cmap="RdYlGn")


plot_evi2(masked)

### 2.3 Calculating the maximum vegetation index value for each month

Rather than removing scenes with cloud cover, it is valuable to combine multiple scenes into a single representative scene.
For calculating the trend in land productivity, the metric used is the maximum vegetation index value.

The following cell applies a resampling operation to group scenes by month, and then applies the maximum. 

At this time, it is also useful to load the relevant data into memory. 
This is done by using the `.compute()` method on the lazy loaded data. 
To see the processing, click the Dask Dashboard URL that was generated at the start of the notebook.

In [ ]:
# Create a monthly maximum now we have masked data
max_values = masked.resample(time="1MS").max()

# Get Dask to run processing
max_evi2 = max_values.evi2.compute()

The next step is to display the monthly maximum index value over time and look for changes.

In [ ]:
max_evi2.plot.imshow(col="time", col_wrap=4, cmap="RdYlGn")

### 2.4  Identifying the period of peak productivity by looking at the average maximum value for the area in each month

Rather than looking at the spatial information, it is possible to summarise the data further into an average value for the whole area in each month. 
This helps identify the general patterns in the vegetation index, such as the period of peak productivity.

In [ ]:
max_evi2.mean(["x", "y"]).plot(ylim=(0, 1))

The time series shows that the index is lowest during September, and the index is at a peak between December and February. 
From this, we can select the peak productivity period as December to February.
Using observations from a three month period will also likely help mitigate the cloud cover that appears frequently in January.

## Part 3: Land productivity trend from 2013 to 2024

The important measure of land productivity in terms of land degredation is related to how it is trending through time.
The purpose of identifying the peak productivity period is to then load data from many years during the same period and compare these over time.

### 3.1: Querying NASA's Earthdata STAC to identify relevent scenes from the peak productivity period of each year since 2013

The first step is to identify all scenes that were captured between the start of December and the end of February for a given year. 
This is done by constructing a loop in the cell below, starting at 2013 (earliest available HLS data) and going through to 2024.

The loop identifies all scenes in each year and appends them to the list of items.

In [ ]:
year_range = range(2013, 2024)

peak_vegetation_items = []

for year in year_range:
    start_date = f"{year}-12-01"
    end_date = f"{year+1}-02-28"
    date_range = f"{start_date}/{end_date}"

    # Search for items in the collection
    items = list(
        stac_client.search(
            collections=collections, intersects=aoi_geom, datetime=date_range
        ).items()
    )

    peak_vegetation_items.extend(items)

print(f"Found {len(peak_vegetation_items)} items")

### 3.2 Loading the data from the identified scenes

As in Part 1, the next step is to use `odc.stac` to lazily load the queried data.

Apart from the use of `peak_vegetation_items` as the list of items to load, all other arguments keep the same values as the loading step used in Part 1.

In [ ]:
trend_data = load(
    peak_vegetation_items,
    resolution=30,
    crs="EPSG:5530",
    chunks={},
    groupby="solar_day",
    stac_cfg=hls_config,
    bands=["red", "green", "blue", "nir", "fmask"],
    geopolygon=aoi,
)

trend_data

### 3.3 Applying masking and calculating the vegetation index

As in Part 2, it is important to apply the quality mask to the data before calculating the index.
This section reuses the functions for masking and calculating the index that were used in Part 2.

In [ ]:
trend_data_masked = apply_quality_mask(trend_data)

trend_data_masked["evi2"] = calculate_evi2(trend_data_masked)

### 3.4 Measuring the maximum vegetation index value for the peak period of each year

The next step is to calculate the maximum value for each three month period, starting in December.
This step first uses the `.resample()` method to group data by quarters, starting in December, then calculates the maximum. 
In practice, the data are grouped by Dec-Jan-Feb, Mar-Apr-May, Jun-Jul-Aug, and Sep-Oct-Nov, and the maximum function is applied to each group. 
Note that the query has already restricted the STAC items to be in the Dec-Jan-Feb quarter.

The second step takes the resampled data and selects all the quarters that begin in December, dropping the additional quarters that contain no data.

The final step is to calculate the maximum for each pixel in each December quarter and load the data into memory using the `.compute()` method

In [ ]:
# Resample data to quarters, from beginning of December
quarters_max = trend_data_masked.resample(time="QS-DEC").max()

# Only keep quaryers where the month start is DEC
dec_quarters_max = quarters_max.sel(time=quarters_max.time.dt.month == 12)

# Compute
dec_quaters_max_evi2 = dec_quarters_max.evi2.compute()

In [ ]:
dec_quaters_max_evi2.plot.imshow(col="time", col_wrap=4, cmap="RdYlGn")

### 3.5 Observing the trend

As in Part 2, it is useful to take the average over all pixels to see the time series. 
In this case, it is useful to use a scatter plot to see whether there is an overall trend in the index value over time.

In [ ]:
dec_quaters_max_evi2.mean(["x", "y"]).plot.scatter(ylim=(0, 1))

## Conclusion and next steps

This notebook demonstrated how to stream data direct from NASA's Earthdata STAC and perform an Earth observation analysis of land productivity, one of the key subindicators of SDG indicator 15.3.1 relating to measuring land degredation. 

The next steps from here would be to identify whether there is a statistically significant trend in the annual maximum EVI2 values, which could then be classified as the land becoming degraded or not degraded. This would then feed into SDG indicator 15.3.1 along with measures on land cover and carbon stocks to produce an overall assessment of the amount of degraded area as a proporition of the total area. 

As a user of this notebook, you may like to:
1. Identify a new area of interest and re-run the analysis
2. Extend the analysis to fit a trend line to the EVI2 over time

You can learn more about how Earth observation can be used for SDG Indicator 15.3.1 from [Satellite Data Requirements for SDGIndicator 15.3.1](https://ceos.org/sdg/files/supportsheets/SDG_15.3), authored by the Committee on Earth Observation Satellites.